In [1]:
######### txt2img by SD XL model & refiner ##########
######### Generate pattern for conceptial art & garment / fabric pattern ##########

from diffusers import DiffusionPipeline
import torch
import matplotlib.pyplot as plt
import PIL
from PIL import Image

########## Run if for seamless tiling ############

def patch_conv(cls):
	init = cls.__init__
	def __init__(self, *args, **kwargs):
		return init(self, *args, **kwargs, padding_mode='circular')
	cls.__init__ = __init__

patch_conv(torch.nn.Conv2d)


#### load both base & refiner models #####
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

##### Parameter settings ######
n_steps = 60
high_noise_frac = 0.8
num_images_perprompt = 1


#### Prompt for pattern generation ####
prompt = "(Damask and dragon pattern:1.3), (Chinese in style:1.1), (Lunar New Year in color tone:1.2), (Absolute Symmetric:1.1), Masterpiece, best quality, ultra-detailed, sharp focus and clear, good contrast, even studio lighting"
negative_prompt = "deformed, disfigured, duplicate, error, extra arms, extra fingers, extra legs, extra limbs, fused fingers"
img = base(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_images_per_prompt=num_images_perprompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images

for i in img:
    img = refiner(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_images_perprompt,
        num_inference_steps=n_steps,
        denoising_start=high_noise_frac,
        image=img,
    ).images
    
#### Show the generated image ####
import PIL
from PIL import Image

for i in img:
    display(i)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [3]:
######### Save the prefered image ###########
img = img[0]
img.save("C:/Users/benle/Desktop/Pattern editing/Trial/"+"XL_Damask_Dragon.png")